In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
df = pd.read_csv('labeled_data.csv')
df.drop(columns=df.columns[0], axis = 1, inplace = True)

In [4]:
df
#  0 -> Hate speech, 1-> Offensive, 2-> Neither

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...
24778,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,6,0,6,0,1,youu got wild bitches tellin you lies


In [59]:
main_df = df[['tweet','class']]
main_df

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
24779,"you've gone and broke the wrong heart baby, an...",2
24780,young buck wanna eat!!.. dat nigguh like I ain...,1
24781,youu got wild bitches tellin you lies,1


In [60]:
main_df.iloc[0,0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [65]:
def preprocess_token(token):
    return token.lemma_.strip().lower()
def valid_token(token):
    return bool(token and token.lemma_.isalnum() and str(token).strip() and not token.is_punct)


In [66]:
word_list = list(df['tweet'])

In [67]:
temp = list()
for i in word_list:
    doc = nlp(i)
    temp.append([preprocess_token(token) for token in doc if valid_token(token)])

In [68]:
new_temp = []
for i in temp:
    new_temp.append(' '.join(i))

In [69]:
main_df['processed_tweet'] = new_temp

/tmp/ipykernel_54309/4128648944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['processed_tweet'] = new_temp


In [70]:
main_df

,tweet,class,processed_tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...,2,rt as a woman you should not complain about cl...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1,rt boy dat cold tyga dwn bad for cuffin dat ho...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,rt dawg rt you ever fuck a bitch and she start...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1,rt she look like a tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1,rt the shit you hear about i might be true or ...
...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1,you be a lie right his tl be trash 8230 now mi...
24779,"you've gone and broke the wrong heart baby, an...",2,you go and break the wrong heart baby and driv...
24780,young buck wanna eat!!.. dat nigguh like I ain...,1,young buck wanna eat dat nigguh like i be not ...
24781,youu got wild bitches tellin you lies,1,youu get wild bitch tellin you lie


In [72]:
main_df.to_csv('clean_data.csv')